In [1]:
'''
Autor: Matheus Silva
Data: Maio 2022
Este projeto recebe um nome de arquivo csv da database
do governo federal sobre a série histórica do preço
dos combustíveis e edita esse arquivo, agrupando
as informações do valor de venda da gasolina por 
estados e região.
'''
# importando bibliotecas
import logging
import pandas as pd
import matplotlib.pyplot as plt

# configurando o logging
logging.basicConfig(
    filename='./results.log',
    level=logging.INFO,
    filemode='w',
    format='%(name)s - %(levelname)s - %(message)s')

In [2]:
filepath = 'data/serie_historica_combustiveis/ca-2021-02.csv'

In [3]:
# lendo csv
def read_data(file_path):
    """Read data from csv.
    Args:
      file_path (str): file path to read.
    Return:
        df_file (DataFrame): returns the file read as a dataframe.
    """
    try:
        df = pd.read_csv(file_path, sep = ';', encoding='ISO-8859-1')
        return df
    except: # pylint: disable=bare-except
        logging.error("We were not able to find %s", file_path)

In [4]:
precos_combustiveis = read_data(filepath)
print(precos_combustiveis.info())
precos_combustiveis.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472856 entries, 0 to 472855
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Regiao - Sigla     472856 non-null  object 
 1   Estado - Sigla     472856 non-null  object 
 2   Municipio          472856 non-null  object 
 3   Revenda            472856 non-null  object 
 4   CNPJ da Revenda    472856 non-null  object 
 5   Nome da Rua        472856 non-null  object 
 6   Numero Rua         472664 non-null  object 
 7   Complemento        96633 non-null   object 
 8   Bairro             471737 non-null  object 
 9   Cep                472856 non-null  object 
 10  Produto            472856 non-null  object 
 11  Data da Coleta     472856 non-null  object 
 12  Valor de Venda     472856 non-null  object 
 13  Valor de Compra    0 non-null       float64
 14  Unidade de Medida  472856 non-null  object 
 15  Bandeira           472856 non-null  object 
dtypes:

,Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Valor de Compra,Unidade de Medida,Bandeira
0,NE,CE,MARACANAU,BEZERRA & MENDES COMERCIAL DE PETRÓLEO LTDA.,05.397.086/0001-51,RODOVIA MARANGUAPE - FORTALEZA - KM 06,S/N,NaN,PARQUE LUZARDO VIANA,61910-000,GASOLINA ADITIVADA,01/07/2021,"5,699",NaN,R$ / litro,IPIRANGA
1,NE,CE,MARACANAU,BEZERRA & MENDES COMERCIAL DE PETRÓLEO LTDA.,05.397.086/0001-51,RODOVIA MARANGUAPE - FORTALEZA - KM 06,S/N,NaN,PARQUE LUZARDO VIANA,61910-000,GASOLINA,01/07/2021,"5,499",NaN,R$ / litro,IPIRANGA
2,NE,CE,MARACANAU,BEZERRA & MENDES COMERCIAL DE PETRÓLEO LTDA.,05.397.086/0001-51,RODOVIA MARANGUAPE - FORTALEZA - KM 06,S/N,NaN,PARQUE LUZARDO VIANA,61910-000,DIESEL S10,01/07/2021,"4,699",NaN,R$ / litro,IPIRANGA
3,NE,CE,MARACANAU,BEZERRA & MENDES COMERCIAL DE PETRÓLEO LTDA.,05.397.086/0001-51,RODOVIA MARANGUAPE - FORTALEZA - KM 06,S/N,NaN,PARQUE LUZARDO VIANA,61910-000,ETANOL,01/07/2021,"5,2",NaN,R$ / litro,IPIRANGA
4,NE,CE,MARACANAU,LUIZA GLAURIA R T MENEZES,03.602.329/0001-10,ESTRADA FORTALEZA MARANGUAPE,S/N,NaN,KAGADO,61901-410,GASOLINA,01/07/2021,"5,49",NaN,R$ / litro,VIBRA ENERGIA


In [5]:
precos_combustiveis = precos_combustiveis.rename(columns={precos_combustiveis.columns[0]: "Regiao - Sigla"})

In [6]:
#coluna 'Valor de Venda' para float
precos_combustiveis['Valor de Venda'] = precos_combustiveis['Valor de Venda'].str.replace(',','.').astype(float)
#selecionando apenas gasolina
preco_gasolina = precos_combustiveis[precos_combustiveis['Produto'] == 'GASOLINA']
print(preco_gasolina.shape)
preco_gasolina.head()

(119166, 16)


,Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Valor de Compra,Unidade de Medida,Bandeira
1,NE,CE,MARACANAU,BEZERRA & MENDES COMERCIAL DE PETRÓLEO LTDA.,05.397.086/0001-51,RODOVIA MARANGUAPE - FORTALEZA - KM 06,S/N,NaN,PARQUE LUZARDO VIANA,61910-000,GASOLINA,01/07/2021,5.499,NaN,R$ / litro,IPIRANGA
4,NE,CE,MARACANAU,LUIZA GLAURIA R T MENEZES,03.602.329/0001-10,ESTRADA FORTALEZA MARANGUAPE,S/N,NaN,KAGADO,61901-410,GASOLINA,01/07/2021,5.490,NaN,R$ / litro,VIBRA ENERGIA
8,NE,CE,MARACANAU,POSTO BOTO LTDA,03.020.995/0001-40,AVENIDA PADRE JOSE HOLANDA DO VALE,809,NaN,PIRATININGA,61905-000,GASOLINA,01/07/2021,5.499,NaN,R$ / litro,RAIZEN
12,NE,CE,MARACANAU,SILVEIRA PETROLEO LTDA,02.349.905/0001-05,AVENIDA OSORIO DE PAIVA,8003,NaN,SIQUEIRA II,61901-410,GASOLINA,01/07/2021,5.650,NaN,R$ / litro,BRANCA
15,CO,GO,CALDAS NOVAS,AUTO POSTO JARDIM BELVEDERE LTDA,06.066.360/0001-72,AVENIDA ELIAS BUFAICAL,S/N,QUADRA41,JARDIM BELVEDERE,75696-320,GASOLINA,01/07/2021,5.832,NaN,R$ / litro,BRANCA


In [7]:
#agrupando por estados e calculando a média do valor de venda
preco_gasolina_estados = preco_gasolina.groupby('Estado - Sigla')[['Valor de Venda']].mean().round(2).reset_index()
preco_gasolina_estados = preco_gasolina_estados.rename(columns = {'Estado - Sigla': 'Estado_Sigla', 'Valor de Venda': 'Preco_Media'})

nova_linha = {'Estado_Sigla': 'Total', 'Preco_Media': round(preco_gasolina['Valor de Venda'].mean(), 2)}
preco_gasolina_estados = preco_gasolina_estados.append(nova_linha, ignore_index = True)

preco_gasolina_estados

,Estado_Sigla,Preco_Media
0,AC,6.76
1,AL,6.37
2,AM,6.30
3,AP,5.47
4,BA,6.40
5,CE,6.36
6,DF,6.58
7,ES,6.40
8,GO,6.64
9,MA,6.15


In [9]:
#agrupando por regiões e calculando a média do valor de venda
preco_gasolina_regioes = preco_gasolina.groupby('Regiao - Sigla')[['Valor de Venda']].mean().round(2).reset_index()
preco_gasolina_regioes = preco_gasolina_regioes.rename(columns = {'Regiao - Sigla': 'Regiao_Sigla', 'Valor de Venda': 'Preco_Media'})

nova_linha = {'Regiao_Sigla': 'Total', 'Preco_Media': round(preco_gasolina['Valor de Venda'].mean(), 2)}
preco_gasolina_regioes = preco_gasolina_regioes.append(nova_linha, ignore_index = True)

preco_gasolina_regioes

,Regiao_Sigla,Preco_Media
0,CO,6.46
1,N,6.34
2,NE,6.36
3,S,6.26
4,SE,6.21
5,Total,6.27
